In [5]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

working_folder='/content/drive/MyDrive/AI/TransformerCode/E-commrece/CustmerFeedback.ipynb'

csv_file_path = os.path.join(working_folder, '/content/drive/MyDrive/AI/TransformerCode/E-commrece/productreview.zip.zip')
df = pd.read_csv(csv_file_path)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,sentiment
0,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2016-12-26T00:00:00.000Z,Purchased on Black FridayPros - Great Price (e...,Powerful tablet,Positive
1,Amazon - Echo Plus w/ Built-In Hub - Silver,Amazon,"Amazon Echo,Smart Home,Networking,Home & Tools...","Electronics,Hardware",2018-01-17T00:00:00.000Z,I purchased two Amazon in Echo Plus and two do...,Amazon Echo Plus AWESOME,Positive
2,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Amazon Echo,Virtual Assistant Speakers,Electro...","Electronics,Hardware",2017-12-20T00:00:00.000Z,Just an average Alexa option. Does show a few ...,Average,Neutral
3,"Fire HD 10 Tablet, 10.1 HD Display, Wi-Fi, 16 ...",Amazon,"eBook Readers,Fire Tablets,Electronics Feature...","Office Supplies,Electronics",2017-08-04T00:00:00.000Z,"very good product. Exactly what I wanted, and ...",Greattttttt,Positive
4,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-01-23T00:00:00.000Z,This is the 3rd one I've purchased. I've bough...,Very durable!,Positive


In [6]:
df = df[["reviews.text","sentiment"]]

In [7]:
def map_sentiment_to_class(sentiment):

    if sentiment=="Negative":
        return 0

    elif sentiment=="Neutral":
        return 1

    elif sentiment=="Positive":
        return 2

df.loc[:, "Class"] = df["sentiment"].apply(map_sentiment_to_class)

In [8]:
total_rows=300

class_rows = int(total_rows/3)

df_class_0 = df[df['Class'] == 0]
df_class_1 = df[df['Class'] == 1]
df_class_2 = df[df['Class'] == 2]

df_sample_0 = df_class_0.sample(n=min(class_rows, len(df_class_0)), random_state=42)
df_sample_1 = df_class_1.sample(n=min(class_rows, len(df_class_1)), random_state=42)
df_sample_2 = df_class_2.sample(n=min(class_rows, len(df_class_2)), random_state=42)

df_final = pd.concat([df_sample_0, df_sample_1, df_sample_2])

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name_pretrained = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name_pretrained)

model = AutoModelForSequenceClassification.from_pretrained(model_name_pretrained)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [10]:
sentence = "I hate this movie"

tokenized_input = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
print(tokenized_input)

output = model(**tokenized_input)
print(output)

logits = output.logits

predicted_class = torch.argmax(logits).item()
print(predicted_class)

{'input_ids': tensor([[   0,  100, 4157,   42, 1569,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 3.2048, -0.9183, -2.2059]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0


In [11]:
sentence = "I am student"

tokenized_input = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

output = model(**tokenized_input)

logits = output.logits

predicted_class = torch.argmax(logits).item()
print(predicted_class)

1


In [12]:
sentence = "The movie is excellent"

tokenized_input = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

output = model(**tokenized_input)

logits = output.logits

predicted_class = torch.argmax(logits).item()
print(predicted_class)

2


In [13]:
def get_accuracy_pt(model, tokenizer, df_param):

    df = df_param.copy()

    df["PredClass"] = 0

    for index, row in df.iterrows():

        sentence = row["reviews.text"]

        tokenized_input = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

        output = model(**tokenized_input)

        logits = output.logits

        predicted_class = torch.argmax(logits).item()

        df.at[index, "PredClass"] = predicted_class

    correct_predictions = (df["Class"] == df["PredClass"]).sum()

    total_predictions = len(df)

    accuracy = correct_predictions / total_predictions
    return round(100*accuracy,2)

In [14]:
acc = get_accuracy_pt(model, tokenizer, df_final)